In [1]:
# Celda 1: Importar y Configurar
from google.cloud import bigquery
import polars as pl

PROJECT_ID = "logistics-prod-analysis"
DATASET_ID = "logistica_prod"
TABLE_ID = "rutas_clean"

TABLE_FULL_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)
print(f"Cliente listo. Consultando tabla: {TABLE_FULL_ID}")


Cliente listo. Consultando tabla: logistics-prod-analysis.logistica_prod.rutas_clean


In [2]:
# Celda 2: Definir la consulta SQL Analítica (¡El requisito clave!)
# Esta es la consulta que le mostrarás al entrevistador

# Celda 2: Definir la consulta SQL Analítica (¡CORREGIDA!)

sql_query = f"""
-- 1. CTE para agregar paquetes por día y conductor
WITH DriverDailyPerformance AS (
  SELECT
    date,
    driver_id,
    SUM(n_paquetes) AS paquetes_del_dia
  FROM
    `{TABLE_FULL_ID}`
  WHERE
    date BETWEEN '2025-01-01' AND '2025-12-31'
  GROUP BY
    date, driver_id
),

-- 2. (NUEVA CTE) Calcular el total de paquetes de cada conductor
DriverTotalPerformance AS (
  SELECT
    driver_id,
    SUM(paquetes_del_dia) AS total_paquetes_driver
  FROM
    DriverDailyPerformance
  GROUP BY
    driver_id
),

-- 3. (MODIFICADA) Unir los datos diarios con los totales
DriverAnalytics AS (
  SELECT
    daily.date,
    daily.driver_id,
    daily.paquetes_del_dia,
    total.total_paquetes_driver, -- Traemos el total calculado

    -- Función de Ventana #1: Media móvil de 7 días (sin cambios)
    AVG(daily.paquetes_del_dia) OVER (
      PARTITION BY daily.driver_id
      ORDER BY daily.date
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS media_movil_7d_paquetes
  FROM
    DriverDailyPerformance AS daily
  JOIN
    DriverTotalPerformance AS total ON daily.driver_id = total.driver_id
)

-- 4. Selección final (MODIFICADA)
SELECT
  date,
  driver_id,
  paquetes_del_dia,
  ROUND(media_movil_7d_paquetes, 2) AS media_movil_7d_paquetes,
  ROUND(paquetes_del_dia - media_movil_7d_paquetes, 2) AS diff_vs_media_movil,

  -- Función de Ventana #2: Ranking (AHORA ES SIMPLE)
  -- Ahora podemos rankear por la columna 'total_paquetes_driver'
  RANK() OVER (
    ORDER BY total_paquetes_driver DESC
  ) AS driver_rank_general

FROM
  DriverAnalytics
WHERE
  driver_id IS NOT NULL
ORDER BY
  driver_rank_general, date DESC;
"""

print("Consulta SQL compleja (corregida) definida.")
print("Consulta SQL compleja definida.")


Consulta SQL compleja (corregida) definida.
Consulta SQL compleja definida.


In [3]:
# Celda 3: Ejecutar y traer el resultado a Polars
print("Ejecutando consulta en BigQuery (esto usa 1TB de cuota gratuita)...")

# .to_arrow() es la forma más rápida de traer datos a Polars/Pandas
df_results_arrow = client.query(sql_query).to_arrow()

# Convertir a Polars para análisis local
df_results_polars = pl.from_arrow(df_results_arrow)

print("¡Consulta completada y resultados traídos a Polars!")
print(f"Forma del resultado: {df_results_polars.shape}")
display(df_results_polars.head())


Ejecutando consulta en BigQuery (esto usa 1TB de cuota gratuita)...


/home/ubuntu-t430/Desktop/gcp-logistics-pipeline/venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


¡Consulta completada y resultados traídos a Polars!
Forma del resultado: (246802, 6)


date,driver_id,paquetes_del_dia,media_movil_7d_paquetes,diff_vs_media_movil,driver_rank_general
"datetime[μs, UTC]",i64,i64,f64,f64,i64
2025-12-31 00:00:00 UTC,1,103,94.43,8.57,1
2025-12-26 00:00:00 UTC,1,69,89.29,-20.29,1
2025-12-13 00:00:00 UTC,1,99,93.57,5.43,1
2025-12-06 00:00:00 UTC,1,100,93.86,6.14,1
2025-11-26 00:00:00 UTC,1,89,92.86,-3.86,1
